In [157]:
import json
from boto3.dynamodb.types import TypeDeserializer

def ddb_deserialize(r, type_deserializer = TypeDeserializer()):
  return type_deserializer.deserialize({"M": r})

In [162]:
import boto3
import pandas

def ddb_query(query):
  client = boto3.client('dynamodb')
  items = []
  resp = client.execute_statement(Statement=query, Limit=1000)
  items.extend(resp['Items'])
  while 'LastEvaluatedKey' in resp:
    resp = client.execute_statement(Statement=query,Limit=1000,NextToken=resp["NextToken"])
    items.extend(resp['Items'])

  items = [ ddb_deserialize(r) for r in items ]
  return items


In [227]:
query= """SELECT * FROM "Session-e3tq3jp6jjh4xjjpvydbqj64e4-dev" WHERE CONTAINS("name", '3 rotations')"""
# query= """ SELECT * FROM "Session-e3tq3jp6jjh4xjjpvydbqj64e4-dev" """
sessions = ddb_query(query)
sessions_df = pandas.DataFrame(sessions)

sections_query= f"""SELECT * FROM "SessionSection-e3tq3jp6jjh4xjjpvydbqj64e4-dev" WHERE sessionID in {list(sessions_df['id'])} """
sections = ddb_query(sections_query)
sections_df = pandas.DataFrame(sections)

sections_df  = (sections_df.groupby(['sessionID'])
      .agg({'start': lambda x: x.tolist(),'end': lambda x: x.tolist(),})
      .reset_index())

sessions_df = sessions_df.set_index('id').join(sections_df.set_index('sessionID'), rsuffix='-section').explode(['quaternionTimestamp','linearAccerationTimestamp', 'quaternionW',	'quaternionX',	'quaternionY','quaternionZ', 'linearAccerationX', 'linearAccerationZ', 'linearAccerationY',		])
sessions_df['isInSection'] = None

def gen_isInSection(row):
  row['isInSection'] = False
  for i in range(len(row['start'])):
    if row['linearAccerationTimestamp'] > row['start'][i] and row['linearAccerationTimestamp'] < row['end'][i]:
      row['isInSection'] = True
    
sessions_df.apply(gen_isInSection, axis=1)

sessions_df


,__typename,_lastChangedAt,linearAccerationTimestamp,createdAt,name,quaternionW,quaternionX,linearAccerationZ,quaternionY,linearAccerationY,quaternionZ,_version,linearAccerationX,updatedAt,owner,quaternionTimestamp,start,end,isInSection
id,,,,,,,,,,,,,,,,,,,
a55eb931-6161-46f3-9a63-8bdb4bb82b66,Session,1672076069786,1672073388.025,2022-12-26T17:34:29.733Z,3 rotations 2,0.9468138813972473,-0.29967445135116577,-0.01034603826701641,-0.09628232568502426,0.023349663242697716,-0.06684619933366776,1,0.013941043056547642,2022-12-26T17:34:29.733Z,43d1cc9a-eacd-4d09-8c12-7352818673de::jdavis,1672073388.025,"[1672073435.136, 1672073424.407, 1672073429.365]","[1672073439.622, 1672073428.162, 1672073433.584]",False
a55eb931-6161-46f3-9a63-8bdb4bb82b66,Session,1672076069786,1672073388.035,2022-12-26T17:34:29.733Z,3 rotations 2,0.9468017220497131,-0.29988008737564087,-0.013361495919525623,-0.09541510790586472,-0.00700776744633913,-0.06733762472867966,1,0.002568020485341549,2022-12-26T17:34:29.733Z,43d1cc9a-eacd-4d09-8c12-7352818673de::jdavis,1672073388.035,"[1672073435.136, 1672073424.407, 1672073429.365]","[1672073439.622, 1672073428.162, 1672073433.584]",False
a55eb931-6161-46f3-9a63-8bdb4bb82b66,Session,1672076069786,1672073388.044,2022-12-26T17:34:29.733Z,3 rotations 2,0.9468564987182617,-0.2999213933944702,-0.026921620592474937,-0.09429247677326202,0.004855335224419832,-0.06796253472566605,1,-0.001068023033440113,2022-12-26T17:34:29.733Z,43d1cc9a-eacd-4d09-8c12-7352818673de::jdavis,1672073388.044,"[1672073435.136, 1672073424.407, 1672073429.365]","[1672073439.622, 1672073428.162, 1672073433.584]",False
a55eb931-6161-46f3-9a63-8bdb4bb82b66,Session,1672076069786,1672073388.054,2022-12-26T17:34:29.733Z,3 rotations 2,0.9468923211097717,-0.2999725639820099,-0.029160894453525543,-0.09336627274751663,-0.002815807471051812,-0.06851374357938766,1,-0.004541176371276379,2022-12-26T17:34:29.733Z,43d1cc9a-eacd-4d09-8c12-7352818673de::jdavis,1672073388.054,"[1672073435.136, 1672073424.407, 1672073429.365]","[1672073439.622, 1672073428.162, 1672073433.584]",False
a55eb931-6161-46f3-9a63-8bdb4bb82b66,Session,1672076069786,1672073388.0640001,2022-12-26T17:34:29.733Z,3 rotations 2,0.946969747543335,-0.29994627833366394,-0.05499688908457756,-0.09225156903266907,0.009926949627697468,-0.06906677037477493,1,-0.0022843487095087767,2022-12-26T17:34:29.733Z,43d1cc9a-eacd-4d09-8c12-7352818673de::jdavis,1672073388.0640001,"[1672073435.136, 1672073424.407, 1672073429.365]","[1672073439.622, 1672073428.162, 1672073433.584]",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53178433-928e-4343-9d9e-4f73db29b702,Session,1672076074989,1672073387.9750009,2022-12-26T17:34:34.938Z,3 rotations,0.9478248953819275,-0.29609933495521545,0.03004973754286766,-0.09934590011835098,0.006611483171582222,-0.06390275061130524,1,0.002835767576470971,2022-12-26T17:34:34.938Z,43d1cc9a-eacd-4d09-8c12-7352818673de::jdavis,1672073387.9750009,"[1672073373.291, 1672073377.854, 1672073382.538]","[1672073376.758, 1672073381.37, 1672073386.261]",False
53178433-928e-4343-9d9e-4f73db29b702,Session,1672076074989,1672073387.9850008,2022-12-26T17:34:34.938Z,3 rotations,0.9477168321609497,-0.29645276069641113,0.022088749334216118,-0.09914669394493103,0.0013293275842443109,-0.06417574733495712,1,0.0035160640254616737,2022-12-26T17:34:34.938Z,43d1cc9a-eacd-4d09-8c12-7352818673de::jdavis,1672073387.9850008,"[1672073373.291, 1672073377.854, 1672073382.538]","[1672073376.758, 1672073381.37, 1672073386.261]",False
53178433-928e-4343-9d9e-4f73db29b702,Session,1672076074989,1672073387.994001,2022-12-26T17:34:34.938Z,3 rotations,0.9476103782653809,-0.29684212803840637,0.01458837278187275,-0.09876777231693268,0.004912371281534433,-0.06453070789575577,1,0.00545943807810545,2022-12-26T17:34:34.938Z,43d1cc9a-eacd-4d09-8c12-7352818673de::jdavis,1672073387.994001,"[1672073373.291, 1672073377.854, 1672073382.538]","[1672073376.758, 1672073381.37, 1672073386.261]",False
